In [1]:
import pandas as pd
X = pd.read_csv('~/data/train_input_Z61KlZo.csv') 


/tmp/ipykernel_7912/31312324.py:2: DtypeWarning: Columns (16,17,29,30,31,126,128,129,132,133,135,138,371) have mixed types. Specify dtype option on import or set low_memory=False.
  X = pd.read_csv('~/data/train_input_Z61KlZo.csv')


In [5]:
Y = pd.read_csv('~/data/train_output_DzPxaPY.csv')

In [6]:
Y.head()

,ID,FREQ,CM,ANNEE_ASSURANCE,CHARGE
0,1,0.0,0.0,1.000000,0.0
1,2,0.0,0.0,1.000000,0.0
2,3,0.0,0.0,0.402740,0.0
3,4,0.0,0.0,0.246575,0.0
4,5,0.0,0.0,0.838356,0.0


In [27]:
y_freq = Y["FREQ"]
y_CM = Y["CM"]

In [25]:
for col in X.select_dtypes(include=["object", "category"]).columns:
    X[col] = X[col].astype("category").cat.codes

In [33]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
y_train_freq = Y_train["FREQ"]
y_test_freq = Y_test["FREQ"]

y_train_CM = Y_train["CM"]
y_test_CM = Y_test["CM"]

model_freq = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100)
model_freq.fit(X_train, y_train_freq)



XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [41]:
import xgboost as xgb

model_CM = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=10)

model_CM.set_params(eval_metric="rmse")  

model_CM.fit(
    X_train, y_train_CM,
    eval_set=[(X_train, y_train_CM), (X_test, y_test_CM)],  
    verbose=True  
)


[0]	validation_0-rmse:6512.96633	validation_1-rmse:6862.67042
[1]	validation_0-rmse:6407.57552	validation_1-rmse:6863.64483
[2]	validation_0-rmse:6305.67530	validation_1-rmse:6866.26749
[3]	validation_0-rmse:6203.09168	validation_1-rmse:6868.72079
[4]	validation_0-rmse:6070.06019	validation_1-rmse:6871.06398
[5]	validation_0-rmse:5979.92880	validation_1-rmse:6877.43141
[6]	validation_0-rmse:5943.03458	validation_1-rmse:6880.35083
[7]	validation_0-rmse:5884.91732	validation_1-rmse:6887.35420
[8]	validation_0-rmse:5832.08916	validation_1-rmse:6904.83319
[9]	validation_0-rmse:5777.69651	validation_1-rmse:6908.13220


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=10, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [46]:
y_pred_freq = model_freq.predict(X_test)
rmse = root_mean_squared_error(y_test_freq, y_pred_freq)
print(f"FREQ : root mean Squared Error: {rmse}")

y_pred_CM = model_CM.predict(X_test)
rmse = root_mean_squared_error(y_test_CM, y_pred_CM)
print(f"CM : Root mean Squared Error: {rmse}")

y_pred_charge = y_pred_freq * y_pred_CM * Y_test["ANNEE_ASSURANCE"]
rmse = root_mean_squared_error(Y_test["CHARGE"], y_pred_charge)
print(f"Charge : Root mean Squared Error: {rmse}")

FREQ : root mean Squared Error: 0.6907877996857297
CM : Root mean Squared Error: 6908.1322390542755
Charge : Root mean Squared Error: 7045.16248518728


In [47]:
y_pred_charge

80050     124.617336
34634       0.038519
63168       0.351036
134612      0.534795
238463      1.245300
             ...    
89382      -0.003506
125169      1.838025
257834      0.880984
317470      0.989026
130950     -0.387443
Name: ANNEE_ASSURANCE, Length: 76722, dtype: float64

In [45]:
Y

,ID,FREQ,CM,ANNEE_ASSURANCE,CHARGE
0,1,0.0,0.0,1.000000,0.0
1,2,0.0,0.0,1.000000,0.0
2,3,0.0,0.0,0.402740,0.0
3,4,0.0,0.0,0.246575,0.0
4,5,0.0,0.0,0.838356,0.0
...,...,...,...,...,...
383605,383606,0.0,0.0,1.000000,0.0
383606,383607,0.0,0.0,1.000000,0.0
383607,383608,0.0,0.0,1.000000,0.0
383608,383609,0.0,0.0,1.000000,0.0


In [29]:
y_pred

array([0.01701383, 0.01022671, 0.00641813, ..., 0.01386836, 0.00862069,
       0.00535143], shape=(76722,), dtype=float32)

In [12]:
X

,ID,ACTIVIT2,VOCATION,TYPERS,ANCIENNETE,ADOSS,CARACT1,CARACT2,CARACT3,INDEM1,...,NBJRR100_MMAX_A,NBJRR100_MSOM_A,RR_VOR_MM_A,RR_VOR_MMAX_A,RRAB_VOR_MM_A,RRAB_VOR_MMAX_A,ANNEE_ASSURANCE,ESPINSEE,AN_EXERC,ZONE
0,1,ACT1,VOC6,1,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,ANNEE5,3
1,2,ACT1,VOC6,1,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,ANNEE5,3
2,3,ACT1,VOC6,1,2,N,R,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,0.402740,NaN,ANNEE5,3
3,4,ACT1,VOC6,2,0,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,0.246575,NaN,ANNEE5,3
4,5,ACT1,VOC6,2,1,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,03. <= 102,03. <= 232,04. >= 25,02. <= 57,0.838356,NaN,ANNEE5,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383605,383606,ACT5,VOC6,2,5,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,01. <= 63,01. <= 137,02. <= 19,01. <= 41,1.000000,ESP1,ANNEE6,95
383606,383607,ACT5,VOC6,2,11,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,01. <= 63,01. <= 137,02. <= 19,01. <= 41,1.000000,ESP1,ANNEE6,95
383607,383608,ACT1,VOC6,1,10,N,N,NaN,NaN,N,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,ESP3,ANNEE6,95
383608,383609,ACT1,VOC6,2,0,N,N,NaN,NaN,N,...,01. <= 0,01. <= 0,02. <= 79,02. <= 176,02. <= 19,01. <= 41,1.000000,ESP1,ANNEE6,95


In [16]:
y_test

80050     0.923288
34634     0.200000
63168     1.000000
134612    0.328767
238463    0.896175
            ...   
89382     0.071038
125169    0.928767
257834    0.553425
317470    1.000000
130950    0.246575
Name: ANNEE_ASSURANCE, Length: 76722, dtype: float64